In [42]:
from datetime import date

import numpy as np
import bermuda as tri
import altair as alt
alt.renderers.enable("html")

RendererRegistry.enable('html')

In [43]:
big_triangle = tri.binary_to_triangle("data/plotting_triangle.trib")

### Data Completeness Plot

This is the **first plot** I reach for when loading a new triangle.  
It helps visualize:  
- The **overall shape** of the triangle  
- Whether any **data is missing**  
- Missing **entire cells** or just **specific fields** within certain periods


In [44]:
big_triangle.plot_data_completeness()

alt.Chart(...)

#### Clipping
Let's clip off some of that older data to make this triangle more manageable.

In [45]:
triangle = big_triangle.clip(min_period=date(2010, 1, 1))

In [46]:
triangle.plot_data_completeness()

alt.Chart(...)

### Triangle Heatmap

Some actuaries still prefer seeing a **table of numbers** in the classic triangle format — maybe they just **miss Excel**.  
For them, we provide a **heatmap** that displays **multiple metrics** while preserving the familiar triangle shape.


In [8]:
triangle.plot_heatmap()

alt.LayerChart(...)

#### Plotting Metrics
In general, these plots can show a few pre-computed metrics.
- "Paid Loss Ratio"
- "Reported Loss Ratio"
- "Incurred Loss Ratio"
- "Paid Loss"
- "Reported Loss"
- "Incurred Loss"
- "Paid ATA"
- "Reported ATA"
- "Paid Incremental ATA"
- "Reported Incremental ATA"

In [47]:
triangle.plot_heatmap("Reported Loss Ratio")

alt.LayerChart(...)

In [48]:
triangle.plot_heatmap("Paid ATA")

alt.LayerChart(...)

### Boxplot of ATA Factors

A better way to visualize **ATA factors** is with a **boxplot**.  
This shows:  
- The **median**  
- The **interquartile range (IQR)**  
- The **minimum and maximum** values


In [13]:
triangle.plot_atas('Reported ATA')

alt.LayerChart(...)

### Right Edge Plot

The **right edge** of a triangle represents data as of the **latest evaluation date**.  
This plot shows:  
- **Earned premium**, **paid loss ratio**, and **reported loss ratio** for each period  
- A typical **gap between paid and reported losses**, often wider in later periods  
- A **drop-off in recent loss ratios** due to limited development  
- Including **earned premium** helps identify periods where low volume drives higher loss ratio volatility


In [14]:
triangle.plot_right_edge()

alt.LayerChart(...)

### Mountain Plot

The **mountain plot** shows **loss development over time**.  
- Each line represents an **iso-development lag**.  
- **X-axis** → Accident or policy period.  
- All periods start at **dev lag 0**, but only the earliest periods reach the longest lags.  
- Loss ratios typically **increase with development lag** and **converge toward an ultimate loss ratio**.  
- **Peaks** indicate periods with **higher loss ratios**; **valleys** show lower ones.  
- Ideally, development patterns remain **consistent over time**, resulting in **parallel lines**.


In [50]:
triangle.plot_mountain()

alt.LayerChart(...)

### Hose Plot

The **hose plot** visualizes **incremental losses** over time  
(up until now, we've been looking at **cumulative values**).  

- Works even though the underlying triangle stores **cumulative cells**  
- **Incremental losses** typically **rise**, then **decline**, eventually approaching **zero**  
- Think of a **hose spraying water**:  
  - The peak of the spray = highest incremental losses  
  - Where it hits the ground ≈ **ultimate loss**


In [56]:
triangle.plot_hose()

alt.LayerChart(...)

### Ballistics Plot

The **ballistics plot** shows **reported loss ratios** and **paid loss ratios** together, with dot colors representing **development lags**.  

- **Reported losses** are usually **higher** than **paid losses** early on, then the two converge.  
- When **reported = paid** (reserves ≈ $0), we've reached the **ultimate loss**.  
- Think of a **projectile** launched toward the **dashed line of equality**:  
  - Each line represents a **period**  
  - Where it intersects the dashed line = the **ultimate loss ratio**

In [57]:
triangle.plot_ballistic()

alt.LayerChart(...)

# Multi-Slice Plots

Suppose you're an insurer covering three lines of business, **Commercial Auto**, **General Liability**, and **Private Auto**. You'll likely have three separate triangles for each, and you might like to view each of them separately, or combine them all together. A single triangle object can have multiple **slices**, or cuts by metadata such as line of business.

In [58]:
ca = tri.binary_to_triangle("data/ca.trib")
gl = tri.binary_to_triangle("data/gl.trib")
pa = tri.binary_to_triangle("data/pa.trib")

combined = pa + ca + gl

### Multi-Slice Plots

When a triangle contains **multiple slices**, plots are **automatically faceted by slice** by default,  
making it easy to compare patterns across different segments.


In [60]:
combined.plot_data_completeness()

alt.ConcatChart(...)

# Customizing Plots
The plots returned by these funcions are Altair LayerChart objects. You can manipulate these objects to customize them to suit your needs.

In [61]:
mountain = triangle.plot_mountain()
mountain

alt.LayerChart(...)

### Modifying Plot Attributes

You can **directly modify plot object properties** to customize visuals —  
for example, updating the **title**, **background color**, and other styling options.


In [62]:
mountain.background = 'palegoldenrod'
mountain.title = "New Custom Name"
mountain.padding = {"left":40}
mountain

alt.LayerChart(...)

You can also use the Altair `.properties` method to modify similar options.

In [65]:
triangle.plot_mountain().properties(title="Other Method")

alt.LayerChart(...)

### Fine-Tuned Plot Styling

For **more precise control** over plot appearance, use the `.configure` methods.  
This approach allows for **advanced styling options** and greater customization.


In [66]:
triangle.plot_right_edge().configure(
    background="#eeeeee",
 ).configure_title(
    font="monospace"
 ).configure_axisX(
    titleColor="blue",
    labelColor="green",
    tickColor="red",
    labelFontSize=10,
).configure_legend(
    direction="horizontal",
    orient="bottom",
).configure_bar(
    stroke="black",
)

alt.LayerChart(...)


### Plotting Stochastic Predictions

At Ledger, we use **Bayesian modeling techniques**, often storing **n posterior samples** in each triangle cell  
instead of relying on single point estimates.

Bermuda’s plots **natively support** these samples and provide multiple ways to **visualize uncertainty**:  
- **Ribbons** — show the range of outcomes smoothly  
- **Segments** — display sample variability directly


In [67]:
rng = np.random.default_rng(1234)
prediction_evals = triangle.evaluation_dates[-3:]
triangle_predictions = triangle.derive_fields(
    paid_loss = lambda cell: (
       rng.normal(cell["paid_loss"], 1e6 * cell.dev_lag(), size=1000)
       if cell.evaluation_date in prediction_evals
       else cell["paid_loss"]
    ),
    reported_loss = lambda cell: (
       rng.normal(cell["reported_loss"], 1e6 * cell.dev_lag(), size=1000)
       if cell.evaluation_date in prediction_evals
       else cell["reported_loss"]
    ),
)

triangle_predictions.plot_right_edge()

alt.LayerChart(...)

In [68]:
triangle_predictions.plot_right_edge(uncertainty_type="segments")

alt.LayerChart(...)

### Built-In Uncertainty Support

All Bermuda plot types **natively support uncertainty visualization**  
when working with **posterior samples** — no extra configuration required.


In [69]:
triangle_predictions.plot_growth_curve()

alt.LayerChart(...)

### Faceting for Uncertainty Visualization

When uncertainty makes it difficult to compare **multiple periods**,  
a common approach is to use **slice faceting**:  

- Create a **multi-slice triangle** where each slice represents a **single period**  
- Plots are then **faceted by period**, making comparisons clearer  
- You also have full control over **plot shapes** and **facet layouts**


In [70]:
triangle_predictions.derive_metadata(
    id = lambda cell: cell.period_start
).plot_growth_curve(width=250, height=200, ncols=2)

alt.ConcatChart(...)

### Fixing Axes Across Facets

By default, each facet has its **own axis limits**, which can make comparisons harder.  
Using Altair’s **`resolve_axis`** method, you can **fix the axes across facets**  
for a more consistent and comparable visualization.


In [27]:
triangle_predictions.derive_metadata(
    id = lambda cell: cell.period_start
).plot_growth_curve(
    width=150, height=100, ncols=5,
    facet_titles=[str(period[0].year) for period in triangle_predictions.periods],
    uncertainty_type="spaghetti",
).resolve_scale(
    x="shared", y="shared", color="independent",
).resolve_axis(
    y="shared"
).properties(
    background="#eeeeee",
).configure_concat(
    spacing=2,
).configure_title(
    offset=1,
).interactive()

alt.ConcatChart(...)

### Custom Metrics

In addition to Bermuda’s **built-in metrics**, you can plot **custom metrics** derived from triangle data.  

- Any **function of the cell fields** can be visualized  
- For example, you might define a custom metric for **reserves**  
- Use the **`metrics_spec`** argument:  
  - **Key** → Axis label  
  - **Value** → Callable (e.g., a `lambda` function to compute reserves for each cell)


In [71]:
triangle.plot_growth_curve(metric_spec = {"Reserves" : lambda cell: cell['reported_loss'] - cell['paid_loss']})

alt.LayerChart(...)

# Try it yourself!

### Problem 1
Try plotting the same reserves plot above, but show reserves as a percentage of earned premium on the y axis.

### Answer 1

In [30]:
triangle.plot_growth_curve(metric_spec = {"Reserves as % of Earned Premium" : lambda cell: 100 * (cell['reported_loss'] - cell['paid_loss']) / cell['earned_premium']})

alt.LayerChart(...)

### Problem 2

Now show the same reserves as a percentage of earned premium in a heatmap

### Answer 2

In [79]:
triangle.plot_heatmap(metric_spec = {"Reserves as % of Earned Premium" : lambda cell: 100 * (cell['reported_loss'] - cell['paid_loss']) / cell['earned_premium']})

alt.LayerChart(...)

## Task

There is an excel file with triangle data called "data/interesting_triangle_example.xlsx". Use bermuda to load the triangle, and create some plots of the data. There's something interesting about it, see if you can find out what...

## Answer

In [31]:
import pandas as pd
raw_data = pd.read_excel("data/excel/interesting_triangle_example.xlsx", sheet_name=None)

In [32]:
example_tri = tri.array_triangle_builder(list(raw_data.values()), ['paid_loss', 'reported_loss', 'earned_premium'])

In [33]:
example_tri.plot_mountain('Reported Loss')

alt.LayerChart(...)

In [34]:
example_tri.plot_growth_curve("Reported Loss")

alt.LayerChart(...)

In [35]:
example_tri.plot_ballistic()

alt.LayerChart(...)

In [38]:
example_tri.plot_atas("Reported ATA")

alt.LayerChart(...)

In [39]:
example_tri

Cumulative Triangle 


 Number of slices:  1 
 Number of cells:  120 
 Triangle category:  Regular 
 Experience range:  2005-01-01/2019-12-31 
 Experience resolution:  12 
 Evaluation range:  2005-12-31/2019-12-31 
 Evaluation resolution:  12 
 Dev Lag range:  0.0 - 168.0 months 
 Fields: 
   earned_premium
   paid_loss
   reported_loss
 Common Metadata: 
   risk_basis  Accident

In [40]:
example_tri.to_long_csv("~/Desktop/example_long.csv")

In [41]:
!open ./data/excel